This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# xarray CfRadial1 backend

In this example, we read CfRadial1 data files using the `xradar` `cfradial1` backend.

Data is also exported to ODIM_H5 and CfRadial2.

In [ ]:
import wradlib as wrl
import warnings

warnings.filterwarnings("ignore")
import matplotlib.pyplot as pl
import numpy as np
import xradar as xd
import datatree as xt
import xarray as xr

try:
    get_ipython().run_line_magic("matplotlib inline")
except:
    pl.ion()

## Load CfRadial1 Volume Data

We use the functionality provided now by ``xradar`` to read the CfRadial1 data into a DataTree.

In [ ]:
fpath = "netcdf/cfrad.20080604_002217_000_SPOL_v36_SUR.nc"
f = wrl.util.get_wradlib_data_file(fpath)
vol = xd.io.open_cfradial1_datatree(f)

### Inspect RadarVolume

In [ ]:
display(vol)

### Inspect root group

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
vol.root

### Inspect sweep group(s)

The sweep-groups can be accessed via their respective keys. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
display(vol["sweep_0"])

### Georeferencing

``sweep_mode`` is assigned coordinate, as we need it available on the DataArray. We have to properly decode it too.

In [ ]:
swp = vol["sweep_0"].ds
swp = swp.assign_coords(sweep_mode=swp.sweep_mode)
swp["sweep_mode"].data = swp["sweep_mode"].item().decode()
swp = swp.pipe(wrl.georef.georeference_dataset)
display(swp)

### Plotting

In [ ]:
swp.DBZ.plot.pcolormesh(x="x", y="y")
pl.gca().set_aspect("equal")

In [ ]:
fig = pl.figure(figsize=(10, 10))
swp.DBZ.wrl.vis.plot(proj="cg", fig=fig)

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
pm = swp.DBZ.wrl.vis.plot(proj=map_proj)
ax = pl.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp.DBZ.wrl.vis.plot(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
import cartopy.feature as cfeature


def plot_borders(ax):
    borders = cfeature.NaturalEarthFeature(
        category="physical", name="coastline", scale="10m", facecolor="none"
    )
    ax.add_feature(borders, edgecolor="black", lw=2, zorder=4)


map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)

DBZ = swp.DBZ
pm = DBZ.where(DBZ > 0).wrl.vis.plot(ax=ax)
plot_borders(ax)
ax.gridlines(draw_labels=True)

In [ ]:
import matplotlib.path as mpath

theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values,
    central_longitude=swp.longitude.values,
)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
ax.set_boundary(circle, transform=ax.transAxes)

pm = swp.DBZ.wrl.vis.plot(proj=map_proj, ax=ax)
ax = pl.gca()
ax.gridlines(crs=map_proj)

In [ ]:
fig = pl.figure(figsize=(10, 8))
proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
ax = fig.add_subplot(111, projection=proj)
pm = swp.DBZ.wrl.vis.plot(ax=ax)
ax.gridlines()

In [ ]:
swp.DBZ.wrl.vis.plot()

### Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray has dimensions and coordinates of it's parent dataset. There are attributes connected which are defined by Cf/Radial standard.

In [ ]:
display(swp.DBZ)

### Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('time')` method, which sorts the radials by time.

In [ ]:
swp.DBZ.sortby("time").plot(x="range", y="time", add_labels=False)

In [ ]:
fig = pl.figure(figsize=(5, 5))
pm = swp.DBZ.wrl.vis.plot(proj={"latmin": 33e3}, fig=fig)

### Mask some values

In [ ]:
swp["DBZ"] = swp["DBZ"].where(swp["DBZ"] >= 0)
swp["DBZ"].plot()

### Export to ODIM and CfRadial2

There are some slight issues with xradar v0.1.0, so we need to remove non-sweep groups before export.

In [ ]:
for k in vol.groups[1:]:
    if "sweep" not in k:
        print(k)
        del vol[k[1:]]

In [ ]:
xd.io.to_odim(vol, "cfradial1_as_odim.h5")

In [ ]:
xd.io.to_cfradial2(vol, "cfradial1_as_cfradial2.nc")

### Import again

xradar doesn't have a special `open_cfradial2_datatree` function, using `datatree.open_datatree` instead.

In [ ]:
vola = xd.io.open_odim_datatree("cfradial1_as_odim.h5", first_dim="time")
display(vola)

In [ ]:
volb = xt.open_datatree("cfradial1_as_cfradial2.nc")
display(volb)

In [ ]:
xr.testing.assert_allclose(vol.root.ds, volb.root.ds)

## More CfRadial1 loading mechanisms

### Use `xr.open_dataset` to retrieve explicit group

<div class="alert alert-warning">

**Warning** <br>
    
Since $\omega radlib$ version 2.0 all xarray backend related functionality is imported from [xradar](https://github.com/openradar/xradar)-package.
    
</div>


In [ ]:
swp_b = xr.open_dataset(
    f, engine="cfradial1", group="sweep_1", backend_kwargs=dict(reindex_angle=False)
)
display(swp_b)